## Imports

In [1]:
import cv2
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from glob import glob
from PIL import Image
import os

## Local Phase Quantization (LPQ)

In [2]:

def lpq(img, winSize=3,freqestim=1,mode='nh'):

	# img = cv2.imread(img_filepath,0)
	rho=0.90

	STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
	sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
	sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

	convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

	img=np.float64(img) # Convert np.image to double
	r=(winSize-1)/2 # Get radius from window size
	x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

	if freqestim==1:  #  STFT uniform window
	    #  Basic STFT filters
	    w0=np.ones_like(x)
	    w1=np.exp(-2*np.pi*x*STFTalpha*1j)
	    w2=np.conj(w1)

	## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
	# Run first filters
	filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
	filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
	filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
	filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

	# Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
	freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                     
	                    filterResp2.real, filterResp2.imag,
	                    filterResp3.real, filterResp3.imag,
	                    filterResp4.real, filterResp4.imag])

	## Perform quantization and compute LPQ codewords
	inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
	LPQdesc=((freqResp>0)*(2**inds)).sum(2)

	## Switch format to uint8 if LPQ code np.image is required as output
	if mode=='im':
		LPQdesc=np.uint8(LPQdesc)
		plt.figure(figsize=(20,10))
		plt.subplot(121)
		plt.imshow(img, cmap='gray')
		plt.title("Original image")

		plt.subplot(122)
		plt.imshow(LPQdesc, cmap='gray')
		plt.title("lpq")
		plt.show()


	## Histogram if needed
	if mode=='nh' or mode=='h':
	    LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

	## Normalize histogram if needed
	if mode=='nh':
	    LPQdesc=LPQdesc/LPQdesc.sum()

	return LPQdesc

## Getting All Data

In [119]:
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".png"):
            print(filename)
            img = cv2.imread(os.path.join(directory, filename),0)
            if img is not None:
                images.append(img)
    return images

In [4]:

marehy_image_list = []
lemonada_image_list = []
scheherazade_image_list = []
ibm_image_list = []
marehy_image_list = load_images_from_directory("images/generated_train/Marehy")
lemonada_image_list = load_images_from_directory("images/generated_train/Lemonada")
scheherazade_image_list = load_images_from_directory("images/generated_train/Scheherazade_New")
ibm_image_list = load_images_from_directory("images/generated_train/IBM_Plex_Sans_Arabic")

In [5]:
features = []
classes = []


## Train Images feature extraction

In [6]:
print("--------------Marehy----------------")
for i in range(1,len(marehy_image_list)):
    features.append(lpq(marehy_image_list[i],mode='nh'))
    classes.append(0)

--------------Marehy----------------


In [7]:
print("--------------Lemonada----------------")
for i in range(1,len(lemonada_image_list)):
    features.append(lpq(lemonada_image_list[i],mode='nh'))
    classes.append(1)

--------------Lemonada----------------


In [8]:
print("--------------Scheherazade----------------")
for i in range(1,len(scheherazade_image_list)):
    features.append(lpq(scheherazade_image_list[i],mode='nh'))
    classes.append(2)

--------------Scheherazade----------------


In [9]:
print("--------------IBM----------------")
for i in range(1,len(ibm_image_list)):
    features.append(lpq(ibm_image_list[i],mode='nh'))
    classes.append(3)

--------------IBM----------------


## Fit classifier

In [170]:
test_images = []
test_images = load_images_from_directory("images/train")
features = np.array(features)

1-1.png
1-2.png
1-3.png
2-1.png
2-2.png
2-3.png
3-1.png
3-2.png
3-3.png
4-1.png
4-2.png
4-3.png


#### KNN

In [171]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(features, classes)

KNeighborsClassifier(n_neighbors=11)

In [172]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(knn.predict([test_image]))

--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[1]


#### Decision Tree

In [173]:
dt = DecisionTreeClassifier()
dt.fit(features, classes)


DecisionTreeClassifier()

In [174]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(dt.predict([test_image]))

--------------Test----------------
[1]
--------------Test----------------
[2]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[2]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]


#### SVM

In [175]:
clf = svm.SVC(kernel='poly',degree=3,C=500,gamma=7)
clf.fit(np.array(features),np.array( classes))

SVC(C=500, gamma=7, kernel='poly')

In [176]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(clf.predict([test_image]))

--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]


#### Gaussian Bayes

In [177]:
gnb = GaussianNB()
bayes = gnb.fit(features, classes)

In [178]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(bayes.predict([test_image]))

--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]


#### Minimum Distance Classifier

In [179]:
clf = NearestCentroid()
clf.fit(features, classes)

NearestCentroid()

In [180]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(clf.predict([test_image]))

--------------Test----------------
[1]
--------------Test----------------
[2]
--------------Test----------------
[1]
--------------Test----------------
[0]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[2]
--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[1]


#### Random Forest

In [184]:
clf = RandomForestClassifier(max_depth=8, random_state=20,n_estimators=100)
clf.fit(features, classes)


RandomForestClassifier(max_depth=8, random_state=20)

In [185]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(clf.predict([test_image]))

--------------Test----------------
[3]
--------------Test----------------
[2]
--------------Test----------------
[3]
--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[3]
--------------Test----------------
[2]
--------------Test----------------
[3]
--------------Test----------------
[1]
--------------Test----------------
[0]


## MLP

In [186]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(5, 1), random_state=11)
# clf = MLPClassifier(hidden_layer_sizes=(5,2),max_iter = 300,activation = 'relu', solver = 'adam',alpha=1e-5,)
clf.fit(features, classes)


C:\Users\20112\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(hidden_layer_sizes=(5, 1), random_state=11, solver='lbfgs')

In [187]:
for i in range(1,len(test_images)):
    print("--------------Test----------------")
    test_image = lpq(test_images[i],mode='nh')
    print(clf.predict([test_image]))

--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[3]
--------------Test----------------
[1]
--------------Test----------------
[1]
--------------Test----------------
[1]
